In [4]:
from urllib.request import urlopen
import pandas as pd
import time
from tqdm.notebook import tqdm
import os
def getHTMLRace(race_id_list: list, skip: bool = True):
    """
    netkeiba.comのraceページのhtmlをスクレイピングしてdata/html/raceに保存する関数。
    """
    html_path_list = []
    for race_id in tqdm(race_id_list):
        url = 'https://db.netkeiba.com/race/' + race_id #race_idからurlを作る
        html = urlopen(url).read() #スクレイピング実行
        filename = 'MLOPS/data/html/race/' + race_id + '.bin'
        html_path_list.append(filename)
        if skip and os.path.isfile(filename): #skipがTrueで、かつbinファイルがすでに存在する場合は飛ばす
            print('race_id {} skipped'.format(race_id))
            continue
        with open(filename, 'wb') as f: #保存するファイルパスを指定
            f.write(html) #保存
        time.sleep(1) #相手サーバーに負担をかけないように1秒待機する
    return html_path_list

In [5]:
race_id_list = ['201901010101', '201901010102']

In [6]:
getHTMLRace(race_id_list)

  0%|          | 0/2 [00:00<?, ?it/s]

['MLOPS/data/html/race/201901010101.bin',
 'MLOPS/data/html/race/201901010102.bin']

In [8]:
html_path_list = getHTMLRace(race_id_list)

  0%|          | 0/2 [00:00<?, ?it/s]

race_id 201901010101 skipped
race_id 201901010102 skipped


In [9]:
html_path_list

['MLOPS/data/html/race/201901010101.bin',
 'MLOPS/data/html/race/201901010102.bin']

In [45]:
from bs4 import BeautifulSoup #htmlを加工するパッケージ
import re #正規表現のパッケージ
def getRawDataResults(html_path_list: list):
    """
    raceページのhtmlを受け取って、レース結果テーブルに変換する関数。
    """
    race_results = {}
    for html_path in tqdm(html_path_list):
        with open(html_path, 'rb') as f:
            html = f.read() #保存してあるbinファイルを読み込む
            df = pd.read_html(html)[0] #メインとなるレース結果テーブルデータを取得
            
            soup = BeautifulSoup(html, "html.parser") #htmlをsoupオブジェクトに変換
            #馬ID、騎手IDをスクレイピング
            horse_id_list = []
            horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                "a", attrs={"href": re.compile("^/horse")}
            )
            for a in horse_a_list:
                horse_id = re.findall(r"\d+", a["href"])
                horse_id_list.append(horse_id[0])
            jockey_id_list = []
            jockey_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                "a", attrs={"href": re.compile("^/jockey")}
            )
            for a in jockey_a_list:
                jockey_id = re.findall(r"\d+", a["href"])
                jockey_id_list.append(jockey_id[0])
            df["horse_id"] = horse_id_list
            df["jockey_id"] = jockey_id_list
            #インデックスをrace_idにする
            race_id = re.findall('\d+', html_path)[0]
            df.index = [race_id] * len(df)
            race_results[race_id] = df
    #pd.DataFrame型にして一つのデータにまとめる
    race_results_df = pd.concat([race_results[key] for key in race_results])
    return race_results_df

In [62]:
results = getRawDataResults(html_path_list)

  0%|          | 0/28224 [00:00<?, ?it/s]

IndexError: list index out of range

In [11]:
results

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師,horse_id,jockey_id
201901010101,1,1,1,ゴルコンダ,牡2,54,ルメール,1:48.3,NaN,1.4,1,518(-16),[東] 木村哲也,2017105318,05339
201901010101,2,3,3,プントファイヤー,牡2,54,岩田康誠,1:50.1,大,3.5,2,496(-8),[東] 手塚貴久,2017104612,05203
201901010101,3,4,4,ラグリマスネグラス,牡2,51,団野大成,1:50.9,5,46.6,6,546(+6),[東] 藤沢和雄,2017103879,01180
201901010101,4,8,9,キタノコドウ,牡2,51,菅原明良,1:51.5,3.1/2,56.8,7,458(-8),[東] 高木登,2017106259,01179
201901010101,5,5,5,ネモフィラブルー,牡2,54,川島信二,1:51.7,1.1/2,140.3,9,436(0),[西] 矢作芳人,2017104140,01062
201901010101,6,8,8,マイネルラクスマン,牡2,54,丹内祐次,1:52.1,2.1/2,9.7,3,480(+8),[東] 金成貴史,2017101930,01091
201901010101,7,2,2,サンモンテベロ,牝2,54,黛弘人,1:52.5,2.1/2,114.7,8,450(+2),[東] 中野栄治,2017100184,01109
201901010101,8,7,7,エスカレーション,牝2,54,藤岡佑介,1:52.5,アタマ,26.1,5,448(-4),[東] 高柳瑞樹,2017102953,01093
201901010101,9,6,6,セイウンジュリア,牝2,54,池添謙一,1:52.6,クビ,16.4,4,470(0),[西] 浅見秀一,2017102421,01032
201901010102,1,7,7,イエスサンキュー,牝3,54,加藤祥太,0:59.2,NaN,4.3,4,492(+2),[西] 羽月友彦,2016102221,01156


In [12]:
for html_path in tqdm(html_path_list):
    print(html_path)

  0%|          | 0/2 [00:00<?, ?it/s]

MLOPS/data/html/race/201901010101.bin
MLOPS/data/html/race/201901010102.bin


In [34]:
def getRawDataInfo(html_path_list: list):
    """
    raceページのhtmlを受け取って、レース情報テーブルに変換する関数。
    """
    race_infos = {}
    for html_path in tqdm(html_path_list):
        with open(html_path, 'rb') as f:
            html = f.read() #保存してあるbinファイルを読み込む
            
            soup = BeautifulSoup(html, "html.parser") #htmlをsoupオブジェクトに変換
            #天候、レースの種類、コースの長さ、馬場の状態、日付をスクレイピング
            texts = (
                soup.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
                + soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
            )
            info = re.findall(r'\w+', texts)
            df = pd.DataFrame()
            for text in info:
                if text in ["芝", "ダート"]:
                    df["race_type"] = [text]
                if "障" in text:
                    df["race_type"] = ["障害"]
                if "m" in text:
                    df["course_len"] = [int(re.findall(r"\d+", text)[-1])] #20211212：[0]→[-1]に修正
                if text in ["良", "稍重", "重", "不良"]:
                    df["ground_state"] = [text]
                if text in ["曇", "晴", "雨", "小雨", "小雪", "雪"]:
                    df["weather"] = [text]
                if "年" in text:
                    df["date"] = [text]
            
            #インデックスをrace_idにする
            race_id = re.findall('\d+', html_path)[0]
            df.index = [race_id] * len(df)
            race_infos[race_id] = df
    #pd.DataFrame型にして一つのデータにまとめる
    race_infos_df = pd.concat([race_infos[key] for key in race_infos])
    return race_infos_df

In [14]:
race_infos = getRawDataInfo(html_path_list)

  0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
race_infos

,course_len,weather,race_type,ground_state,date
201901010101,1800,曇,芝,良,2019年7月27日
201901010102,1000,曇,ダート,良,2019年7月27日


In [16]:
def getRawDataResults(html_path_list: list):
    """
    raceページのhtmlを受け取って、レース結果テーブルに変換する関数。
    """
    race_results = {}
    for html_path in tqdm(html_path_list):
        with open(html_path, 'rb') as f:
            html = f.read() #保存してあるbinファイルを読み込む
            df = pd.read_html(html)[0] #メインとなるレース結果テーブルデータを取得
            
            soup = BeautifulSoup(html, "html.parser") #htmlをsoupオブジェクトに変換
            #馬ID、騎手IDをスクレイピング
            horse_id_list = []
            horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                "a", attrs={"href": re.compile("^/horse")}
            )
            for a in horse_a_list:
                horse_id = re.findall(r"\d+", a["href"])
                horse_id_list.append(horse_id[0])
            jockey_id_list = []
            jockey_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                "a", attrs={"href": re.compile("^/jockey")}
            )
            for a in jockey_a_list:
                jockey_id = re.findall(r"\d+", a["href"])
                jockey_id_list.append(jockey_id[0])
            df["horse_id"] = horse_id_list
            df["jockey_id"] = jockey_id_list
            #インデックスをrace_idにする
            race_id = re.findall('(?<=race/)\d+', html_path)[0]
            df.index = [race_id] * len(df)
            race_results[race_id] = df
    #pd.DataFrame型にして一つのデータにまとめる
    race_results_df = pd.concat([race_results[key] for key in race_results])
    return race_results_df

In [17]:
results2 = getRawDataResults(html_path_list)

  0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
results2

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師,horse_id,jockey_id
201901010101,1,1,1,ゴルコンダ,牡2,54,ルメール,1:48.3,NaN,1.4,1,518(-16),[東] 木村哲也,2017105318,05339
201901010101,2,3,3,プントファイヤー,牡2,54,岩田康誠,1:50.1,大,3.5,2,496(-8),[東] 手塚貴久,2017104612,05203
201901010101,3,4,4,ラグリマスネグラス,牡2,51,団野大成,1:50.9,5,46.6,6,546(+6),[東] 藤沢和雄,2017103879,01180
201901010101,4,8,9,キタノコドウ,牡2,51,菅原明良,1:51.5,3.1/2,56.8,7,458(-8),[東] 高木登,2017106259,01179
201901010101,5,5,5,ネモフィラブルー,牡2,54,川島信二,1:51.7,1.1/2,140.3,9,436(0),[西] 矢作芳人,2017104140,01062
201901010101,6,8,8,マイネルラクスマン,牡2,54,丹内祐次,1:52.1,2.1/2,9.7,3,480(+8),[東] 金成貴史,2017101930,01091
201901010101,7,2,2,サンモンテベロ,牝2,54,黛弘人,1:52.5,2.1/2,114.7,8,450(+2),[東] 中野栄治,2017100184,01109
201901010101,8,7,7,エスカレーション,牝2,54,藤岡佑介,1:52.5,アタマ,26.1,5,448(-4),[東] 高柳瑞樹,2017102953,01093
201901010101,9,6,6,セイウンジュリア,牝2,54,池添謙一,1:52.6,クビ,16.4,4,470(0),[西] 浅見秀一,2017102421,01032
201901010102,1,7,7,イエスサンキュー,牝3,54,加藤祥太,0:59.2,NaN,4.3,4,492(+2),[西] 羽月友彦,2016102221,01156


In [2]:
import glob
html_path_list = glob.glob('MLOPS/data/html/race/*.bin')
html_path_list.sort()

In [239]:
html_path_list[1765]

'MLOPS/data/html/race/202110010405.bin'

In [243]:
results = getRawDataResults(html_path_list)

  0%|          | 0/13539 [00:00<?, ?it/s]

In [244]:
results.to_pickle('results.pickle')

In [4]:
from urllib.request import urlopen
import pandas as pd
import time
from tqdm.notebook import tqdm
import os
from bs4 import BeautifulSoup #htmlを加工するパッケージ
import re #正規表現のパッケージ

def getRawDataInfo(html_path_list: list):
    """
    raceページのhtmlを受け取って、レース情報テーブルに変換する関数。
    """
    race_infos = {}
    for html_path in tqdm(html_path_list):
        with open(html_path, 'rb') as f:
            html = f.read() #保存してあるbinファイルを読み込む
            
            soup = BeautifulSoup(html, "html.parser") #htmlをsoupオブジェクトに変換
            #天候、レースの種類、コースの長さ、馬場の状態、日付をスクレイピング
            texts = (
                soup.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
                + soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
            )
            info = re.findall(r'\w+', texts)
            df = pd.DataFrame()
            for text in info:
                if text in ["芝", "ダート"]:
                    df["race_type"] = [text]
                if "障" in text:
                    df["race_type"] = ["障害"]
                if "m" in text:
                    df["course_len"] = [int(re.findall(r"\d+", text)[-1])] #20211212：[0]→[-1]に修正
                if text in ["良", "稍重", "重", "不良"]:
                    df["ground_state"] = [text]
                if text in ["曇", "晴", "雨", "小雨", "小雪", "雪"]:
                    df["weather"] = [text]
                if "年" in text:
                    df["date"] = [text]
            
            #インデックスをrace_idにする
            race_id = re.findall('(?<=race/)\d+', html_path)[0]
            df.index = [race_id] * len(df)
            race_infos[race_id] = df
    #pd.DataFrame型にして一つのデータにまとめる
    race_infos_df = pd.concat([race_infos[key] for key in race_infos])
    return race_infos_df

In [5]:
race_infos = getRawDataInfo(html_path_list)

  0%|          | 0/13539 [00:00<?, ?it/s]

In [7]:
race_infos

,course_len,weather,race_type,ground_state,date
201801010101,1800,晴,芝,良,2018年7月28日
201801010102,1000,晴,ダート,良,2018年7月28日
201801010103,2400,晴,ダート,良,2018年7月28日
201801010104,1700,晴,ダート,良,2018年7月28日
201801010105,1500,晴,芝,良,2018年7月28日
...,...,...,...,...,...
202110040808,1000,曇,ダート,稍重,2021年9月5日
202110040809,1800,小雨,芝,良,2021年9月5日
202110040810,1700,小雨,ダート,稍重,2021年9月5日
202110040811,1200,小雨,芝,良,2021年9月5日


In [8]:
race_infos.to_pickle('race_infos.pickle')

In [6]:
def getRawDataReturn(html_path_list: list):
    """
    raceページのhtmlを受け取って、払い戻しテーブルに変換する関数。
    """
    horse_results = {}
    for html_path in tqdm(html_path_list):
        with open(html_path, 'rb') as f:
            html = f.read() #保存してあるbinファイルを読み込む
            
            html = html.replace(b'<br />', b'br')
            dfs = pd.read_html(html)
            #dfsの1番目に単勝〜馬連、2番目にワイド〜三連単がある
            df = pd.concat([dfs[1], dfs[2]])
            
            race_id = re.findall('(?<=race/)\d+', html_path)[0]
            df.index = [race_id] * len(df)
            horse_results[race_id] = df
    #pd.DataFrame型にして一つのデータにまとめる
    horse_results_df = pd.concat([horse_results[key] for key in horse_results])
    return horse_results_df

In [9]:
return_tables = getRawDataReturn(html_path_list)

  0%|          | 0/13539 [00:00<?, ?it/s]

In [10]:
return_tables

,0,1,2,3
201801010101,単勝,1,110,1
201801010101,複勝,1br6,100br130,1br2
201801010101,馬連,1 - 6,180,1
201801010101,ワイド,1 - 6br1 - 3br3 - 6,110br110br150,1br2br3
201801010101,馬単,1 → 6,210,1
...,...,...,...,...
202110040812,馬連,3 - 11,2270,9
202110040812,ワイド,3 - 11br4 - 11br3 - 4,660br190br260,9br1br2
202110040812,馬単,11 → 3,3520,14
202110040812,三連複,3 - 4 - 11,1200,3


In [12]:
return_tables.to_pickle('return_tables.pickle')

In [1]:
import glob
horse_id_list = glob.glob('MLOPS/data/html/horse/*.bin')
horse_id_list.sort()

In [2]:
horse_id_list

['MLOPS/data/html/horse/2006106754.bin',
 'MLOPS/data/html/horse/2007100107.bin',
 'MLOPS/data/html/horse/2007100828.bin',
 'MLOPS/data/html/horse/2007106312.bin',
 'MLOPS/data/html/horse/2008100175.bin',
 'MLOPS/data/html/horse/2008100889.bin',
 'MLOPS/data/html/horse/2008101384.bin',
 'MLOPS/data/html/horse/2008102492.bin',
 'MLOPS/data/html/horse/2008103002.bin',
 'MLOPS/data/html/horse/2008103192.bin',
 'MLOPS/data/html/horse/2008104268.bin',
 'MLOPS/data/html/horse/2008104812.bin',
 'MLOPS/data/html/horse/2008104835.bin',
 'MLOPS/data/html/horse/2008110132.bin',
 'MLOPS/data/html/horse/2009100335.bin',
 'MLOPS/data/html/horse/2009100359.bin',
 'MLOPS/data/html/horse/2009100502.bin',
 'MLOPS/data/html/horse/2009100509.bin',
 'MLOPS/data/html/horse/2009100515.bin',
 'MLOPS/data/html/horse/2009100667.bin',
 'MLOPS/data/html/horse/2009100680.bin',
 'MLOPS/data/html/horse/2009100706.bin',
 'MLOPS/data/html/horse/2009100957.bin',
 'MLOPS/data/html/horse/2009100967.bin',
 'MLOPS/data/htm

In [3]:
from urllib.request import urlopen
import pandas as pd
import time
from tqdm.notebook import tqdm
import os
from bs4 import BeautifulSoup #htmlを加工するパッケージ
import re #正規表現のパッケージ

def getRawDataHorseResults(html_path_list: list):
    """
    horseページのhtmlを受け取って、馬の過去成績のDataFrameに変換する関数。
    """
    horse_results = {}
    for html_path in tqdm(html_path_list):
        with open(html_path, 'rb') as f:
            html = f.read() #保存してあるbinファイルを読み込む
            
            df = pd.read_html(html)[3]
            #受賞歴がある馬の場合、3番目に受賞歴テーブルが来るため、4番目のデータを取得する
            if df.columns[0]=='受賞歴':
                df = pd.read_html(html)[4]
                
            horse_id = re.findall('(?<=horse/)\d+', html_path)[0]
            
            df.index = [horse_id] * len(df)
            horse_results[horse_id] = df
            
    #pd.DataFrame型にして一つのデータにまとめる
    horse_results_df = pd.concat([horse_results[key] for key in horse_results])
    return horse_results_df

In [4]:
horse_results = getRawDataHorseResults(horse_id_list)

  0%|          | 0/25634 [00:00<?, ?it/s]

In [5]:
horse_results

,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,...,着差,ﾀｲﾑ指数,通過,ペース,上り,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金
2006106754,2018/03/24,3中山1,晴,11.0,日経賞(G2),NaN,15.0,5.0,8,625.9,...,5.5,**,14-14-14-15,31.0-36.0,38.0,452(-6),NaN,NaN,ガンコ,NaN
2006106754,2018/02/17,1東京7,晴,11.0,ダイヤモンドS(G3),NaN,14.0,7.0,11,403.8,...,6.8,**,14-14-14-14,36.9-35.8,40.0,458(+6),NaN,NaN,フェイムゲーム,NaN
2006106754,2018/01/07,1京都2,晴,11.0,万葉S(OP),NaN,9.0,3.0,3,115.1,...,2.8,**,9-9-9-9,36.7-35.8,37.1,452(-2),NaN,NaN,トミケンスラーヴァ,NaN
2006106754,2017/12/02,5中山1,晴,11.0,ステイヤーズS(G2),NaN,10.0,7.0,8,133.0,...,11.1,**,10-10-10-10,37.6-35.5,43.0,454(-10),NaN,NaN,アルバート,NaN
2006106754,2017/02/18,1東京7,曇,11.0,ダイヤモンドS(G3),NaN,15.0,7.0,13,154.8,...,2.8,**,15-15-14-14,38.3-34.4,35.7,464(0),NaN,NaN,アルバート,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019110123,2022/01/29,1東京1,曇,4.0,3歳1勝クラス,NaN,16.0,1.0,1,9.2,...,1.4,**,4-3,34.7-37.9,38.9,552(+4),NaN,NaN,リッキーマジック,NaN
2019110123,2021/12/19,6中京6,曇,10.0,寒椿賞(1勝クラス),NaN,16.0,8.0,15,9.7,...,0.6,**,2-2,33.6-37.8,38.3,548(+4),NaN,NaN,グットディール,101.0
2019110123,2021/11/27,5東京7,晴,9.0,カトレアS(OP),NaN,15.0,8.0,15,7.2,...,0.5,**,4-5,34.9-37.3,37.5,544(+2),NaN,NaN,コンシリエーレ,240.0
2019110123,2021/10/31,4東京8,曇,2.0,2歳未勝利,NaN,11.0,2.0,2,3.1,...,0.0,**,1-1,36.8-34.4,34.4,542(-4),NaN,NaN,(ビヨンドザタイム),355.0


In [6]:
horse_results.to_pickle('horse_results.pickle')

In [19]:
from urllib.request import urlopen
import pandas as pd
import time
from tqdm.notebook import tqdm
import os
from bs4 import BeautifulSoup #htmlを加工するパッケージ
import re #正規表現のパッケージ

def getRawDataPeds(html_path_list: list):
    """
    horse/pedページのhtmlを受け取って、血統のDataFrameに変換する関数。
    """
    peds = {}
    for html_path in tqdm(html_path_list):
        with open(html_path, 'rb') as f:
            html = f.read() #保存してあるbinファイルを読み込む
            
            df = pd.read_html(html)[0]
            #重複を削除して1列のSeries型データに直す
            generations = {}
            horse_id = re.findall('(?<=ped/)\d+', html_path)[0]
            for i in reversed(range(5)):
                generations[i] = df[i]
                df.drop([i], axis=1, inplace=True)
                df = df.drop_duplicates()
            ped = pd.concat([generations[i] for i in range(5)]).rename(horse_id)
            peds[horse_id] = ped.reset_index(drop=True)
    #pd.DataFrame型にして一つのデータにまとめる
    peds_df = pd.concat([peds[key] for key in peds], axis=1).T.add_prefix('peds_')
    return peds_df

In [20]:
import glob
html_path_list_ped = glob.glob('MLOPS/data/html/ped/*.bin')
html_path_list_ped.sort()

In [4]:
html_path_list_ped

['MLOPS/data/html/ped/2006106754.bin',
 'MLOPS/data/html/ped/2007100107.bin',
 'MLOPS/data/html/ped/2007100828.bin',
 'MLOPS/data/html/ped/2007106312.bin',
 'MLOPS/data/html/ped/2008100175.bin',
 'MLOPS/data/html/ped/2008100889.bin',
 'MLOPS/data/html/ped/2008101384.bin',
 'MLOPS/data/html/ped/2008102492.bin',
 'MLOPS/data/html/ped/2008103002.bin',
 'MLOPS/data/html/ped/2008103192.bin',
 'MLOPS/data/html/ped/2008104268.bin',
 'MLOPS/data/html/ped/2008104812.bin',
 'MLOPS/data/html/ped/2008104835.bin',
 'MLOPS/data/html/ped/2008110132.bin',
 'MLOPS/data/html/ped/2009100335.bin',
 'MLOPS/data/html/ped/2009100359.bin',
 'MLOPS/data/html/ped/2009100502.bin',
 'MLOPS/data/html/ped/2009100509.bin',
 'MLOPS/data/html/ped/2009100515.bin',
 'MLOPS/data/html/ped/2009100667.bin',
 'MLOPS/data/html/ped/2009100680.bin',
 'MLOPS/data/html/ped/2009100706.bin',
 'MLOPS/data/html/ped/2009100957.bin',
 'MLOPS/data/html/ped/2009100967.bin',
 'MLOPS/data/html/ped/2009100970.bin',
 'MLOPS/data/html/ped/200

In [21]:
peds = getRawDataPeds(html_path_list_ped)

  0%|          | 0/25634 [00:00<?, ?it/s]

In [22]:
peds

,peds_0,peds_1,peds_2,peds_3,peds_4,peds_5,peds_6,peds_7,peds_8,peds_9,...,peds_52,peds_53,peds_54,peds_55,peds_56,peds_57,peds_58,peds_59,peds_60,peds_61
2006106754,ゴールドアリュール 1999 栗毛 [血統][産駒] Halo系,ユーモレスク 2000 栗毛 [血統][産駒] FNo.[1-o],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],ニキーヤ Nikiya(米) 1993 鹿毛 [血統][産駒],アフリート Afleet(加) 1984 栗毛 [血統][産駒],アリーウイン Alywin(米) 1984 栗毛 [血統][産駒],Halo 1969 黒鹿毛 [血統][産駒],Wishing Well 1975 鹿毛 [血統][産駒],Nureyev 1977 鹿毛 [血統][産駒],Reluctant Guest 1986 鹿毛 [血統][産駒],...,Green Ticket,Ways to Learn,Native Dancer,Raise You,On-And-On,Plum Cake,Tudor Minstrel,Glen Line,Count Fleet,Honor Bound
2007100107,アドマイヤマックス 1999 鹿毛 [血統][産駒] Halo系,ドリーミングレイナ 1997 黒鹿毛 [血統][産駒] FNo.[12],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],ダイナシュート 1982 栗毛 [血統][産駒],パラダイスクリーク Paradise Creek(米) 1989 黒鹿毛 [血統][産駒],エルレイナ 1990 黒鹿毛 [血統][産駒],Halo 1969 黒鹿毛 [血統][産駒],Wishing Well 1975 鹿毛 [血統][産駒],ノーザンテースト Northern Taste(加) 1971 栗毛 [血統][産駒],シヤダイマイン 1973 黒鹿毛 [血統][産駒],...,Sassafras,Sensibility,Bold Reasoning,My Charmer,Buckpasser,Bayou,Herbager,Delta,Pass,Schooner
2007100828,スウェプトオーヴァーボード Swept Overboard(米) 1997 芦毛 [血統][...,サンシャワーキッス 1995 栗毛 [血統][産駒] FNo.[22-d],エンドスウィープ End Sweep(米) 1991 鹿毛 [血統][産駒],Sheer Ice 1982 芦毛 [血統][産駒],アンバーシャダイ 1977 鹿毛 [血統][産駒],ミルレーサー Millracer(米) 1983 鹿毛 [血統][産駒],フォーティナイナー Forty Niner(米) 1985 栗毛 [血統][産駒],Broom Dance 1979 鹿毛 [血統][産駒],Cutlass 1970 鹿毛 [血統][産駒],Hey Dolly A. 1974 芦毛 [血統][産駒],...,Gallant Man,Europa,Rialto,Wild Violet,Verso,La Rochelle,Intentionally,My Dear Girl,Cornish Prince,Milan Mill
2007106312,ホワイトマズル White Muzzle(英) 1990 鹿毛 [血統][産駒] Lyphard系,セーヌフロウ 1999 鹿毛 [血統][産駒] FNo.[3-d],ダンシングブレーヴ Dancing Brave(米) 1983 鹿毛 [血統][産駒],Fair of the Furze 1982 鹿毛 [血統][産駒],ティンバーカントリー Timber Country(米) 1992 栗毛 [血統][産駒],パリゼット 1989 鹿毛 [血統][産駒],Lyphard 1969 鹿毛 [血統][産駒],Navajo Princess 1974 鹿毛 [血統][産駒],Ela-Mana-Mou 1976 鹿毛 [血統][産駒],Autocratic 1974 鹿毛 [血統][産駒],...,Swaps,Portage,Nearctic,Natalma,Victoria Park,Lady Angela,Ambiorix,Striking,Fleet Nasrullah,Fine Catch
2008100175,デュランダル 1999 栗毛 [血統][産駒] Halo系,サンレイククイン 1996 芦毛 [血統][産駒] FNo.[3-h],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],サワヤカプリンセス 1986 栗毛 [血統][産駒],Cozzene 1980 芦毛 [血統][産駒],レイクワース Lake Worth(米) 1989 鹿毛 [血統][産駒],Halo 1969 黒鹿毛 [血統][産駒],Wishing Well 1975 鹿毛 [血統][産駒],ノーザンテースト Northern Taste(加) 1971 栗毛 [血統][産駒],スコッチプリンセス Scotch Princess(米) 1970 栗毛 [血統][産駒],...,Sir Gaylord,Blue Canoe,Hail to Reason,Bebopper,Dr. Fager,Speedwell,Nijinsky,Fish - Bar,Hail to Reason,Pange
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019110116,American Pharoah (米) 2012 鹿毛 [血統][産駒] Mr. Pros...,Nomee 2015 栗毛 [血統][産駒] FNo.[42],Pioneerof the Nile 2006 黒鹿毛 [血統][産駒],Littleprincessemma 2006 栗毛 [血統][産駒],City Zip 1998 栗毛 [血統][産駒],Miss Utada 2001 栗毛 [血統][産駒],エンパイアメーカー Empire Maker(米) 2000 黒鹿毛 [血統][産駒],Star of Goshen 1994 鹿毛 [血統][産駒],Yankee Gentleman 1999 鹿毛 [血統][産駒],Exclusive Rosette 1993 栗毛 [血統][産駒],...,Tri Jet,Sailaway,Red God,Runaway Bride,Halo,Ballade,Northern Dancer,Height of Fashion,Dara Monarch,Possessive
2019110118,Arrogate (米) 2013 芦毛 [血統][産駒] Mr. Prospector系,Amen Hallelujah 2007 [血統][産駒] FNo.[8-h],Unbridled's Song (米) 1993 芦毛 [血統][産駒],Bubbler 2006 黒鹿毛 [血統][産駒],Montbrook Montbrook(米) 1990 黒鹿毛 [血統][産駒],Sara's Success 1998 [血統][産駒],Unbridled 1987 鹿毛 [血統][産駒],Trolley Song 1983 芦毛 [血統][産駒],Distorted Humor (米) 1993 栗毛 [血統][産駒],Grechelle 1995 黒鹿毛 [血統][産駒],...,Diplomat Way,Old Bess,Super Concorde,Grey Sister,Tunerup,Paris Or Bust,Alydar,Priceless Fame,Well Mannered,Vassar Grad
2019110119,Arrogate (米) 2013 芦毛 [血統][産駒] Mr. Prospector系,Flatter Up 2014 [血統][産駒] FNo.[19-c],Unbridled's Song (米) 1993 芦毛 [血統][産駒],Bubbler 2006 黒鹿毛 [血統][産駒],Flatter 1999 鹿毛 [血統][産駒],Tenacious Tina 2007 [血統][産駒],Unbridled 1987 鹿毛 [血統][産駒],Trolley Song 1983 芦毛 [血統][産駒],Distorted Humor (米) 1993 栗毛 [血統][産駒],Grechelle 1995 黒鹿毛 [血統][産駒],...,Northern Dancer,Glowing Tribute,Raise a Native,Sweet Tooth,Danzig,Cold Hearted,Skywalker,Gentle Hands,Darby Creek Road,Khal On Me
2019110122,Klimt 2014 鹿毛 [血統][産駒] 

In [23]:
peds.to_pickle('peds.pickle')


In [24]:
from urllib.request import urlopen
import pandas as pd
import time
from tqdm.notebook import tqdm
import os
from bs4 import BeautifulSoup #htmlを加工するパッケージ
import re #正規表現のパッケージ

In [27]:
race_id_list = []
# レースIDのリストを作成
for year in range (2021, 2023):
    for place in [30, 35, 36, 42, 43, 44, 45, 46, 47, 48, 50, 51, 54, 55]:
        for month in range(1, 13):
            for day in range(1, 32):
                for race in range(1, 13):
                    race_id = str(year) + str(place).zfill(2) + str(month).zfill(2) + str(day).zfill(2) + str(race).zfill(2)
                    race_id_list.append(race_id)

In [25]:
import time
from tqdm.notebook import tqdm
import os
from urllib.request import urlopen
import pandas as pd
from bs4 import BeautifulSoup #htmlを加工するパッケージ
import re #正規表現のパッケージ
def getHTMLRace(race_id_list: list, skip: bool = True):
    """
    netkeiba.comのraceページのhtmlをスクレイピングしてdata/html/raceに保存する関数。
    """
    html_path_list = []
    for race_id in tqdm(race_id_list):
        url = 'https://db.netkeiba.com/race/' + race_id #race_idからurlを作る
        html = urlopen(url).read() #スクレイピング実行
        filename = 'MLOPS/data/html_nar/race/' + race_id + '.bin'
        html_path_list.append(filename)
        if skip and os.path.isfile(filename): #skipがTrueで、かつbinファイルがすでに存在する場合は飛ばす
            print('race_id {} skipped'.format(race_id))
            continue
        with open(filename, 'wb') as f: #保存するファイルパスを指定
            f.write(html) #保存
        time.sleep(1) #相手サーバーに負担をかけないように1秒待機する
    return html_path_list

In [28]:
getHTMLRace(race_id_list)

  0%|          | 0/124992 [00:00<?, ?it/s]

['MLOPS/data/html_nar/race/202130010101.bin',
 'MLOPS/data/html_nar/race/202130010102.bin',
 'MLOPS/data/html_nar/race/202130010103.bin',
 'MLOPS/data/html_nar/race/202130010104.bin',
 'MLOPS/data/html_nar/race/202130010105.bin',
 'MLOPS/data/html_nar/race/202130010106.bin',
 'MLOPS/data/html_nar/race/202130010107.bin',
 'MLOPS/data/html_nar/race/202130010108.bin',
 'MLOPS/data/html_nar/race/202130010109.bin',
 'MLOPS/data/html_nar/race/202130010110.bin',
 'MLOPS/data/html_nar/race/202130010111.bin',
 'MLOPS/data/html_nar/race/202130010112.bin',
 'MLOPS/data/html_nar/race/202130010201.bin',
 'MLOPS/data/html_nar/race/202130010202.bin',
 'MLOPS/data/html_nar/race/202130010203.bin',
 'MLOPS/data/html_nar/race/202130010204.bin',
 'MLOPS/data/html_nar/race/202130010205.bin',
 'MLOPS/data/html_nar/race/202130010206.bin',
 'MLOPS/data/html_nar/race/202130010207.bin',
 'MLOPS/data/html_nar/race/202130010208.bin',
 'MLOPS/data/html_nar/race/202130010209.bin',
 'MLOPS/data/html_nar/race/2021300

In [31]:
race_id_list = []
# レースIDのリストを作成
for year in range (2020, 2021):
    for place in [30, 35, 36, 42, 43, 44, 45, 46, 47, 48, 50, 51, 54, 55]:
        for month in range(1, 13):
            for day in range(1, 32):
                for race in range(1, 13):
                    race_id = str(year) + str(place).zfill(2) + str(month).zfill(2) + str(day).zfill(2) + str(race).zfill(2)
                    race_id_list.append(race_id)

In [33]:
import time
from tqdm.notebook import tqdm
import os
from urllib.request import urlopen
import pandas as pd
from bs4 import BeautifulSoup #htmlを加工するパッケージ
import re #正規表現のパッケージ
def getHTMLRace(race_id_list: list, skip: bool = True):
    """
    netkeiba.comのraceページのhtmlをスクレイピングしてdata/html/raceに保存する関数。
    """
    html_path_list = []
    for race_id in tqdm(race_id_list):
        url = 'https://db.netkeiba.com/race/' + race_id #race_idからurlを作る
        html = urlopen(url).read() #スクレイピング実行
        filename = 'MLOPS/data/html_nar/race_2020/' + race_id + '.bin'
        html_path_list.append(filename)
        if skip and os.path.isfile(filename): #skipがTrueで、かつbinファイルがすでに存在する場合は飛ばす
            print('race_id {} skipped'.format(race_id))
            continue
        with open(filename, 'wb') as f: #保存するファイルパスを指定
            f.write(html) #保存
        time.sleep(1) #相手サーバーに負担をかけないように1秒待機する
    return html_path_list

In [34]:
getHTMLRace(race_id_list)

  0%|          | 0/62496 [00:00<?, ?it/s]

['MLOPS/data/html_nar/race_2020/202030010101.bin',
 'MLOPS/data/html_nar/race_2020/202030010102.bin',
 'MLOPS/data/html_nar/race_2020/202030010103.bin',
 'MLOPS/data/html_nar/race_2020/202030010104.bin',
 'MLOPS/data/html_nar/race_2020/202030010105.bin',
 'MLOPS/data/html_nar/race_2020/202030010106.bin',
 'MLOPS/data/html_nar/race_2020/202030010107.bin',
 'MLOPS/data/html_nar/race_2020/202030010108.bin',
 'MLOPS/data/html_nar/race_2020/202030010109.bin',
 'MLOPS/data/html_nar/race_2020/202030010110.bin',
 'MLOPS/data/html_nar/race_2020/202030010111.bin',
 'MLOPS/data/html_nar/race_2020/202030010112.bin',
 'MLOPS/data/html_nar/race_2020/202030010201.bin',
 'MLOPS/data/html_nar/race_2020/202030010202.bin',
 'MLOPS/data/html_nar/race_2020/202030010203.bin',
 'MLOPS/data/html_nar/race_2020/202030010204.bin',
 'MLOPS/data/html_nar/race_2020/202030010205.bin',
 'MLOPS/data/html_nar/race_2020/202030010206.bin',
 'MLOPS/data/html_nar/race_2020/202030010207.bin',
 'MLOPS/data/html_nar/race_2020